### 공동주택관리 시스템
* 2018.08, 강남구, 삼성도 소재 모든 아파트,
  아파트 명칭, 법정동 주소,
  아파트 주차대수 (지상, 지하)등을 추출
* 단, 실제 URL에 질의문자열을 포함시켜 정보 추출

### k-apt.go.kr URL분석

1. ~/kaptinfo/openkaptinfo.do
   + 요청 정보가 없음
   
2. getBjdList.do : 요청 폼 데이터 포함 (지역)
   + ajax로 작동, 시도선택시 -> 시군구출력 -> 읍면동 출력
   + ?bjd_gbn = SIDO                 (시도출력)  
   + ?bjd_gbn = SGG&bjd_code = 11    (서울특별시 선택시, 시군구 출력)  
   + ?bjd_gbn = EMD&bjd_code = 11680 (서울특별시 강남구, 읍면동 출력)
   + 읍면동 선택시 bjd_code와 search_date 변수를 생성한 후 getBjdList.do를 호출

3. getBjdList.do : 요청한 폼 데이터 포함 (지역내 모든 아파트)
   + ?bjd_code =11680105&earch_date: 201807
   
4. getKaptSetday.do : 요총 폼 데이터 포함 (아파트 상세정보)
   + ?kapt_code = A10026350
   getKaptInfo_Detail.do

In [19]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
import json
import requests

In [32]:
# 딘지 키워드 검색
year = '2018'
month = '08'
sido = '서울특별시'
gugun = '강남구'
dong = '삼성동'

base = 'http://www.k-apt.go.kr/'
url1 =  base + 'kaptinfo/openkaptinfo.do'
url2 =  base + 'cmmn/bjd/getBidList.do'
url3 =  base + 'kaptinfo/getKaptList.do'
url4 =  base + 'kaptinfo/getKaptSetday.do'
path = "/home/hadoop/chromedriver"

In [33]:
# 셀레니엄 실행
opts = Options()
opts.add_argument('--headless')
chrome = webdriver.Chrome(executable_path=path, options=opts)
chrome.get(url1)
chrome.implicitly_wait(2)

In [34]:
# 검색할 단지 정보 선택
yy = Select(chrome.find_element(By.CLASS_NAME, 'combo_YYYY'))

In [35]:
yy.select_by_visible_text(year)
time.sleep(1)

In [36]:
# 검색할 년도 정보 선택
mm = Select(chrome.find_element(By.CLASS_NAME, 'combo_MM'))
mm.select_by_visible_text(month)
time.sleep(1)

In [37]:
# 검색할 광역시도 정보 선택
sd = Select(chrome.find_element(By.CLASS_NAME, 'combo_SIDO'))
sd.select_by_visible_text(sido)
time.sleep(1)

In [38]:
# 구군
gg = Select(chrome.find_element(By.CLASS_NAME, 'combo_SGG'))
gg.select_by_visible_text(gugun)
time.sleep(1)

In [39]:
# 읍면동
dg = Select(chrome.find_element(By.CLASS_NAME, 'combo_EMD'))
dg.select_by_visible_text(dong)
time.sleep(1)

In [50]:
# 선택한 시구동에 대한 전체코드값txt_bjdcode 출력
# 단, 이 코드값은 숨겨져 있기때문에 요소값을 출력할 수 없음
# 따라서, 자바스크립트로 display:none을 display:block으로 바꿔야 함.
bjd = chrome.find_element(By.CLASS_NAME, 'txt_bjdCode')
chrome.execute_script('arguments[0].style.display="block"',bjd)
print(bjd.text)

11680105


In [ ]:
# 선택한 시 동구에 존재하는 모든 아파트의 코드값 KAPT_CODE 출력
# 단, 이 코드값은 숨겨져 있기때문에 요소값을 출력할 수 없음


In [45]:
td1 = chrome.find_element(By.XPATH, '//tr[@id="1"]/td[1]')
chrome.execute_script('arguments[0].style.display="block"', td1)
print(td1.text)

A10026350


### 삼성동센트럴아이파크 주차장 정보 출력

In [ ]:
kapt_code = td1.text
url4 = url4 + '?kapt_code=' + kapt_code
chrome.get(url4)
chrome.page_source[:100]

#### 삼성동의 모든 아파트 정보 출력

In [40]:
cnt = chrome.find_element(By.CSS_SELECTOR, 'table[id="aptInfoList"] tr:last-child').get_attribute('id')
cnt = int(cnt)
# print(cnt)

15


In [ ]:
for i in range(1, cnt+1):
    code = chrome.find_element(By.XPATH, f'//tr[@id="{i}"]/td[1]')
    chrome.execute_script('arguments[0].style.display="block"', code)
    print(code.text)
    time.sleep(1)

In [47]:
chrome.get(url4)

#### selenium으로 불러온 소스를 json형식으로 다시 읽어옴

In [48]:
html = BeautifulSoup(chrome.page_source)
apt = json.loads(html.find('body').text)


In [49]:
print(apt.get("kaptInfo").get("KAPTD_PCNT"))
print(apt.get("kaptInfo").get("KAPTD_PCNTU"))

0
743


#### '삼성동센트럴아이파크'의 주차장 정보 출력 2

In [56]:
res = requests.get(url4)
html = BeautifulSoup(res.text)
apt2 = json.loads(html.find('body').text)

In [58]:
print(apt2.get("kaptInfo").get("KAPTD_PCNT"))
print(apt2.get("kaptInfo").get("KAPTD_PCNTU"))
print(apt2.get("kaptInfo").get("KAPT_NAME"))

0
743
삼성동센트럴아이파크
